<a href="https://colab.research.google.com/github/thuc-github/MIS710-T12023/blob/main/Week%206/MIS710_Lab6_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **MIS710 Lab 6 Week 6**
Author: Associate Professor Lemai Nguyen

Objectives: 
1. To learn to build and test KNN models for classification and regression
2. To evaluate the models based on the ML problem
3. To optimise k



# **1. Import libraries and functions**

In [ ]:
# import libraries 
import pandas as pd #for data manipulation and analysis
import numpy as np
 
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
!pip install pydotplus #interface for graph visualisation
!pip install graphviz #for graph visualisation

# **2. Case One: Churn Prediction**

**KNN classifier**

Dataset: [Telco Customer Churn] https://www.kaggle.com/datasets/blastchar/telco-customer-churn

**Context**
"Predict behavior to retain customers. You can analyze all relevant customer data and develop focused customer retention programs." [IBM Sample Data Sets]

**Content**
Each row represents a customer, each column contains customer’s attributes described on the column Metadata.

The data set includes information about:

* Customers who left within the last month – the column is called Churn
Services that each customer has signed up for – phone, multiple lines, internet, online security, online backup, device protection, tech support, and streaming TV and movies
* Customer account information – how long they’ve been a customer, contract, payment method, paperless billing, monthly charges, and total charges
* Demographic info about customers – gender, age range, and if they have partners and dependents

**Inspiration**
To explore this type of models and learn more about the subject.

New version from IBM:
https://community.ibm.com/community/user/businessanalytics/blogs/steven-macko/2019/07/11/telco-customer-churn-1113

## **2.1. Loading data**

In [ ]:
url='https://raw.githubusercontent.com/VanLan0/MIS710/main/Customers.csv'

In [ ]:
#loading data
records = pd.read_csv(url)

records.head()

## **2.2. Data Preparation, Exploration and Visualisation**

### **Data cleansing**

* Inspect columns and correct data types
* Detecting and handling missing data


In [ ]:
#Inspect columns and data types to print the following outcome


In [ ]:
#totalcharges is wrongly documented as string
records['TotalCharges'] = records['TotalCharges'].apply(pd.to_numeric, errors='coerce')

In [ ]:
#Inspect missing data, hint .isnull().sum())


In [ ]:
#As the distribution is skewed, replace the missing values with median
records['TotalCharges'].fillna(records['TotalCharges'].median(),inplace=True)


**Question.** Alternatively you can drop the datapoints with missing Total Charges, why and why not?

records.dropna(subset=['TotalCharges'], axis = 0, inplace = True)

In [ ]:
# Remove customer IDs from the data set, hint .drop(['customerID'], axis = 1)


### **EDA**

* Analyse and visualise each variable
* Any strong correlation from the dataset?  
* How to deal with categorical features? 

In [ ]:
#Inspect target variable
records.Churn.value_counts()

In [ ]:
sns.countplot(x=records['Churn'])

In [ ]:
cats=['gender','SeniorCitizen', 'Dependents', 'PhoneService','MultipleLines','InternetService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','Contract','PaperlessBilling','PaymentMethod']
for i in cats:
   print(i, ':\n')
   print(records[i].value_counts())
   print('\n')
   

In [ ]:
for i in cats:
   plt.figure()
   sns.countplot(x=records[i])

Write your own observations

In [ ]:
nums=['tenure', 'MonthlyCharges', 'TotalCharges']
for i in nums:
   print(i, ':\n')
   print(records[i].describe())
   print('\n')


In [ ]:
for i in nums:
  plt.figure()
  #write your own code to display boxplot for (x=records[i])


In [ ]:
for i in nums:
  plt.figure()    
  #write your owncode to display histplot(data=records, x=i,  bins=20, kde=True)


Explore relationships

In [ ]:
for i in cats:
   plt.figure()
   sns.countplot(x=records[i], hue=records['Churn'])

In [ ]:
for i in nums:
  plt.figure()    
  sns.kdeplot(data=records, x=i, hue='Churn')

In [ ]:
sns.clustermap(data=records.corr(), annot=True, cmap='crest')

### **Data preparation**
* Feature selection
* Target specification
* Data spliting

In [ ]:
# Get the categorical columns
cat_columns = records.select_dtypes(include=['object']).columns
cat_columns

In [ ]:
#Convert categorical variables to numerical using get dummies
records=pd.get_dummies(records, columns=cat_columns, drop_first=True)

print(records.info())

In [ ]:
records=records.rename(columns={'Churn_Yes':'Churn'})

In [ ]:
#write your own code to display a heatmap for data=records.corr(), cmap="Blues"

In [ ]:
#Define predictors and label
X=records.drop('Churn', axis=1)
y=records['Churn']


## **2.3. KNN Classifier Model building**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier #Import KNN classifier class
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
#import scaler
from sklearn.preprocessing import StandardScaler

It is a common practice not to scale the target variable (y_train and y_test) because it is not used as an input to the model during training or prediction.

Algorithms such as decision tree regressors, random forests, and support vector regression are generally not very sensitive to differences in scale between the input variables and target variable. 

However, algorithms such as linear regression and neural networks can be more sensitive to scale differences and may require normalization or standardization of the input variables and/or target variable.

If you scale the target variable (y_train and y_test), you would need to perform an inverse transformation on the predicted values to get them back to their original scale.

In [ ]:
# Normalize the features using StandardScaler
scaler = StandardScaler()
X_norm=scaler.fit_transform(X)

In [ ]:
# Split the data into X_train, X_test, y_train, y_test hint: use train_test_split(X_norm, y, test_size=0.35, stratify = y, random_state=2023 )


## **2.4. Performance Evaluation**
* Classification report
* Confusion matrix 
* ROC and AUC

In [ ]:
# Train a KNN model
k = 15 # Number of neighbors
knn = KNeighborsClassifier(n_neighbors=k) #try it with p=1 and p=2

#fit the knn with X_train and y_train

# Make predictions on the testing set



### **Classification report**

In [ ]:
# Print the confusion_matrix and classification report using y_test and y_pred


In [ ]:
#get predicted probabilities for the main class
y_pred_probs_norm = knn.predict_proba(X_test)
y_pred_probs_norm = y_pred_probs_norm[:, 1]
print(y_pred_probs_norm)

### **ROC curve and AUC**

In [ ]:
#get fpr and tpr and plot the ROC curve
from sklearn.metrics import roc_curve, roc_auc_score

fpr, tpr, thresholds = roc_curve(y_test, y_pred_probs_norm)
print('AUC:', '%.3f' % metrics.auc(fpr, tpr))
sns.scatterplot(x=fpr, y=tpr, label='Roc Curve')

In [ ]:
inspection=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred, 'Probability':y_pred_probs_norm})
inspection.head(10)

### **Find the best threshold**
Find the best threshold using the thresholds in ROC curve

Examine the two blocks of code below, which one would you like to use, when and why?

In [ ]:
from sklearn.metrics import accuracy_score
# Find the best threshold based on accuracy
accuracy = []
for threshold in thresholds:
    y_pred_t = [1 if prob >= threshold else 0 for prob in y_pred_probs_norm]
    accuracy.append(accuracy_score(y_test, y_pred_t))
best_threshold = thresholds[accuracy.index(max(accuracy))]

print(best_threshold)

In [ ]:
from sklearn.metrics import f1_score
# Find the best threshold based on F1 score
f1 = []
for threshold in thresholds:
    y_pred_t = [1 if prob >= threshold else 0 for prob in y_pred_probs_norm]
    f1.append(f1_score(y_test, y_pred_t))
best_threshold = thresholds[f1.index(max(f1))]

print(best_threshold)

In [ ]:
#get predicted probabilities for best threshold
y_pred_best = (y_pred_probs_norm >= best_threshold).astype(bool)

print(y_pred_best)
 
 

In [ ]:
#print confusion matrix
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred_best))
print(classification_report(y_test, y_pred_best))

In [ ]:
#what is displayed in the outcomes?
RocCurveDisplay.from_predictions(y_test, y_pred_probs_norm)
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_best)
plt.show()


In [ ]:
#Model evaluation with the original predictions 
print("Accuracy: ", '%.3f' % metrics.accuracy_score(y_test,y_pred))
print("Precision: ", '%.3f' % metrics.precision_score(y_test,y_pred))
print("Recall: ", '%.3f' % metrics.recall_score(y_test,y_pred))
print("F1: ", '%.3f' % metrics.f1_score(y_test,y_pred))


In [ ]:
#Now that we have the y_pred_best for the best threshold, generate the above evaluation results for it 



## **2.5. Optimising k**
based on accuracy

In [ ]:
# Define a list of k values to test
k_values = list(range(1, 41))

# Train and evaluate KNN classifiers with different k values

best_k=0
best_accuracy=0
best_f1=0
accuracy_scores = []
accuracy = 0
error_rate=1-accuracy
error_rates=[]
for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred_k=knn.predict(X_test)
    accuracy = knn.score(X_test, y_test)
    accuracy_scores.append(accuracy)
    error_rates.append(1-accuracy)
    if accuracy > best_accuracy:
        best_k = k
        best_accuracy = accuracy
        best_f1 = metrics.f1_score(y_test, y_pred_k)

# Find the best k value with highest accuracy score
#best_k = k_values[np.argmax(accuracy_scores)]
print(f"Best k value: {best_k}")
print(f"Best accuracy: {best_accuracy:.3f}")
print(f"F1 score for best accuracy: {best_f1:.3f}")

# Plot k values against accuracy scores
#plt.plot(k_values, accuracy_scores, color='red')
plt.xlabel('k')
plt.ylabel('Error rate')
plt.title('Error rates for different k values')
plt.plot(k_values, error_rates, color='blue')
plt.show()

**Optimise k based on f1_score**

In [ ]:
# Define a list of k values to test
k_values = list(range(1, 41))

# Train and evaluate KNN classifiers with different k values
best_k=0
best_f1=0
best_accuracy=0
f1_scores = []
f1 = 0
for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred_k=knn.predict(X_test)
    f1 = metrics.f1_score(y_test, y_pred_k)
    f1_scores.append(f1)
    accuracy_k=metrics.accuracy_score(y_test, y_pred_k)
    if ((f1 > best_f1) ):
        best_k = k
        best_f1 = f1
        best_accuracy = metrics.accuracy_score(y_test,y_pred_k)

# Find the best k value with highest f1 score
#best_k = k_values[np.argmax(accuracy_scores)]
print(f"Best k value: {best_k}")
print(f"Best F1 score: {best_f1:.3f}")
print(f"Accuracy for Best F1 score: {best_accuracy:.3f}")

# Plot k values against accuracy scores
#plt.plot(k_values, accuracy_scores, color='red')
plt.xlabel('k')
plt.ylabel('F1 score')
plt.title('F1 scores for different k values')
plt.plot(k_values, f1_scores, color='blue')
plt.show()

**Try it yourself!**

* Rebuild the model with the optimal k
* Evaluate the model

## **Insight: KNNImputer**

KNNImputer looks at the K nearest neighboring data points that have complete information for a variable with missing and takes an average (or median) of those values to fill in the missing value. The value of K is specified by the user as a hyperparameter.

from sklearn.impute import KNNImputer

**Impute missing data using KNN imputation**
imputer = KNNImputer(n_neighbors=5)
records[['TotalCharges_imputed']] = imputer.fit_transform(records[['TotalCharges']])

**Try it the above code yourself**

Note: KNNImputer cannot be directly applied to categorical data. 

# **3. Case Two: Insurance Premium Estimation**

**KNN regression**

https://www.kaggle.com/datasets/mirichoi0218/insurance 

**Context**
Machine Learning with R by Brett Lantz is a book that provides an introduction to machine learning using R. As far as I can tell, Packt Publishing does not make its datasets available online unless you buy the book and create a user account which can be a problem if you are checking the book out from the library or borrowing the book from a friend. All of these datasets are in the public domain but simply needed some cleaning up and recoding to match the format in the book.

Content - Columns

* age: age of primary beneficiary

* sex: insurance contractor gender, female, male

* bmi: Body mass index, providing an understanding of body, weights that are relatively high or low relative to height,
objective index of body weight (kg / m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9

* children: Number of children covered by health insurance / Number of dependents

* smoker: Smoking

* region: the beneficiary's residential area in the US, northeast, southeast, southwest, northwest.

* charges: Individual medical costs billed by health insurance

**Acknowledgements**
The dataset is available on GitHub here.

**Inspiration**
Can you accurately predict insurance costs?

## **3.1 Loading data**

In [ ]:
url='https://raw.githubusercontent.com/VanLan0/MIS710/main/insurance.csv'


In [ ]:
#loading data
records = pd.read_csv(url)

records.head()

## **3.2. Data Preparation, Exploration and Visualisation**

### **Data cleansing**

* Inspect columns and correct data types
* Detecting and handling missing data


In [ ]:
records.info()

In [ ]:
#Inspect missing data
print(records.isnull().sum())

### **EDA**

* Analyse and visualise each variable
* Any strong correlation from the dataset?  
* How to deal with categorical features? 

In [ ]:
records.describe()

In [ ]:
nums=['age','bmi', 'dependants', 'charges']
for i in nums:
  plt.figure()
  #write code to display kdeplot(data=records, x=i)

In [ ]:
for i in nums:
  plt.figure()
  #write code to display displot for each numerical variable (data=records, x=i, bins=20)

In [ ]:
cats=['sex','smoker', 'region', 'dependants']
for i in cats:
   print(i, ':\n')
   print(records[i].value_counts())
   print('\n')

In [ ]:
for i in cats:
   print(i, ':\n')
   plt.figure()
   sns.countplot(data=records, x=i)

Write your observations of the following graphs

In [ ]:
sns.kdeplot(data=records, x='age', hue='region')

In [ ]:
sns.kdeplot(data=records, x='charges', hue='region')

In [ ]:
#Write your own code to generate the graph below

In [ ]:
sns.scatterplot(data=records, x='age', y='charges', hue='smoker')

In [ ]:
#write your own code to display the below

**Your task:** Write your observation for the graph below:

In [ ]:
sns.boxplot(data=records, x='charges', y='smoker', showmeans=True)

In [ ]:
#write your owncode to display the following

In [ ]:
#visualise clustermap
sns.clustermap(data=records.corr(), cmap="Blues",linewidths=.9, annot=True)

In [ ]:
#Convert categorical variables to numerical using get dummies
records=pd.get_dummies(records, columns=['sex', 'region'], drop_first=True)

print(records.info())

In [ ]:
#convert categorical data to numerical 
def coding_smoking(x):
    if x=='yes': return 1
    if x=='no': return 0
       
records['smoker'] = records['smoker'].apply(coding_smoking)

In [ ]:
records.info()

### **Data preparation**

* Feature selection: X
* Target specification: y
* Scale data
* Data spliting: X_train, X_test, y_train, y_test

In [ ]:
X=records.drop('charges', axis=1)
y=records['charges']

We don't often scale the target variable (y_train and y_test) in regression problems because it is not used as an input to the model during training or prediction. If models are sensitive to scale (eg linear regression) then it is good to scale y as well.

In [ ]:
# Normalize the features using StandardScaler
scaler = StandardScaler()
X_norm=scaler.fit_transform(X)

In [ ]:
#split dataset
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.3)

## **3.3. KNN Model building**

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


In [ ]:
# Create a KNN regressor object
k=5
knn = KNeighborsRegressor(k)

# Fit the model to the training data



## **3.4. Performance Evaluation**
* Root Mean Squared Error (RMSE)measures the differences between predicted and actual values of the target variable. 

* Mean Absolute Error (MAE) measures the average magnitude of the errors between predicted and actual values.

* R-Squared (R²) measures the proportion of variance in the target variable that can be explained by the independent variables - also called Coefficient of Determination. 

In [ ]:
# Predict the house prices for the testing data
y_pred = knn.predict(X_test)

In [ ]:
inspection=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})
inspection.head()

In [ ]:
# Calculate performance metrics
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Print performance metrics
print("Root Mean Squared Error: {:.3f}".format(rmse))
print("R Squared: {:.3f}".format(r2))
print("Absolute Squared Error: {:.3f}".format(mae))

In [ ]:
#Write code to review descriptive stars for the target charges

**Your task:** Write your own interpretation of the model performance

In [ ]:
#Plot residuals, i.e. the differences between the actual and predicted values. 
plt.hist(x=y_test-y_pred, bins=50)
plt.xlabel='error'
plt.ylabels='count'
plt.show()

In [ ]:
#if plt.xlabel() plays up, you can reload plt 
import matplotlib.pyplot as plt
from importlib import reload
plt=reload(plt)

In [ ]:
sns.kdeplot(x=y_test-y_pred)

In [ ]:
# Create a scatter plot of residuals against predicted values
plt.scatter(y_pred, y_test-y_pred)
plt.title('Residual Plot')
plt.ylabel('Residuals')
plt.xlabel('Predicted values')
plt.show()

## **3.5. Optimising k**
based on rmse

In [ ]:
# Define a list of k values to test
k_values = list(range(1, 31))

# Train and evaluate KNN classifiers with different k values
best_k=32
best_rmse=30000000
error_rates=[]
for k in k_values:
    knn = KNeighborsRegressor(k)
    knn.fit(X_train, y_train)
    y_pred=knn.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    error_rates.append(rmse)
    if rmse <= best_rmse:
        best_k = k
        best_rmse = rmse

# Find the best k value with highest accuracy score
print(f"Best k value: {best_k}")
print(f"Best rmse: {best_rmse:.3f}")


In [ ]:
# Plot k values against accuracy scores
plt.ylabel('RMSE')
plt.xlabel('k')
plt.title('Error rates for different k values')
plt.plot(k_values, error_rates, color='blue')
plt.show()

**Your task:** Write your code to optimise k based on another metric

**Try it yourself!**

* Rebuild the model with the optimal k
* Evaluate the model

# **4. Do it yourself**

Now practise what you have learned in this topic with previous datasets that you are familiar with, such as the Titanic, Biopsy and House Price datasets
